## Setup and Import Libraries

In [1]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

In [2]:
_ = load_dotenv(find_dotenv())

In [3]:
client = OpenAI()

## Creating File

In [4]:
filename = "Wonka Chocolate Facility Rules.pdf"

In [5]:
file = client.files.create(
    file=open(file=filename, mode='rb'),
    purpose='assistants'
)

In [6]:
file.id

'file-LDhyz5M15ex1ciUVKuDsEd'

In [7]:
file

FileObject(id='file-LDhyz5M15ex1ciUVKuDsEd', bytes=38968, created_at=1742247405, filename='Wonka Chocolate Facility Rules.pdf', object='file', purpose='assistants', status='processed', expires_at=None, status_details=None)

In [8]:
for file in client.files.list():
    print(file.id, file.filename)

file-LDhyz5M15ex1ciUVKuDsEd Wonka Chocolate Facility Rules.pdf


## Creating Assistant

In [11]:
assistant = client.beta.assistants.create(
    name='Rules Explainer',
    instructions='You answer information about rules based on your knowledge base of PDF files',
    model='gpt-3.5-turbo',
    tools=[{'type':'file_search'}]
)

## Creating Thread and Message

In [12]:
thread = client.beta.threads.create()

In [15]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content='Can I bring my cat to the Wonka Chocolate Facility?',
    attachments=[{"file_id": file.id, "tools": [{"type": "file_search"}]}]
)

## Create Run

In [16]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [17]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
)

In [18]:
run.status

'completed'

## Display Response

In [19]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order='asc'
)

In [20]:
def display_thread_messages(messages):
    for thread_message in messages.data:
        print(thread_message.content[0].text.value)
        print("\n")

In [21]:
display_thread_messages(messages=messages)

Can I bring my cat to the Wonka Chocolate Facility?


Only guide dogs are allowed on the premises of the Wonka Chocolate Facility. Guide dogs must be properly trained and housebroken, owners must be in control of them at all times, and they should not be disruptive or aggressive. Additionally, guide dogs must wear a harness or vest clearly identifying them as guide dogs, and their owners are responsible for cleaning up after them【4:0†source】.




In [23]:
for message in messages:
    print(message)
    print("\n")

Message(id='msg_5sY8gUdiSIek62d1xe8MLi7T', assistant_id=None, attachments=[Attachment(file_id='file-LDhyz5M15ex1ciUVKuDsEd', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Can I bring my cat to the Wonka Chocolate Facility?'), type='text')], created_at=1742248222, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_rkXP1ihNKQQcvaE74l1QpZYL')


Message(id='msg_SVtinjxHuSEOrlPA9Q0CUD4G', assistant_id='asst_mFwoCTFTatTYFdYCpW8udizY', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=391, file_citation=FileCitation(file_id='file-LDhyz5M15ex1ciUVKuDsEd'), start_index=379, text='【4:0†source】', type='file_citation')], value='Only guide dogs are allowed on the premises of the Wonka Chocolate Facility. Guide dogs must be properly train

In [29]:
message.content[0].text.annotations[0]

FileCitationAnnotation(end_index=391, file_citation=FileCitation(file_id='file-LDhyz5M15ex1ciUVKuDsEd'), start_index=379, text='【4:0†source】', type='file_citation')

In [30]:
message.content[0].text.value

'Only guide dogs are allowed on the premises of the Wonka Chocolate Facility. Guide dogs must be properly trained and housebroken, owners must be in control of them at all times, and they should not be disruptive or aggressive. Additionally, guide dogs must wear a harness or vest clearly identifying them as guide dogs, and their owners are responsible for cleaning up after them【4:0†source】.'

## Delete Assistant

In [31]:
response = client.beta.assistants.delete(
    assistant_id=assistant.id
)

In [32]:
response

AssistantDeleted(id='asst_mFwoCTFTatTYFdYCpW8udizY', deleted=True, object='assistant.deleted')